<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Predicting customer churn with IBM Watson Machine Learning</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
  <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/customer-satisfaction-prediction/images/users_banner_2-03.png?raw=true" width="600" alt="Icon"> </th>
   </tr>
</table>

This notebook contains steps and code to develop a predictive model, and start scoring new data. This notebook introduces commands for getting data and for basic data cleaning and exploration, pipeline creation, model training, model persistance to Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.5 and Apache® Spark 2.1.

You will use a data set, **Telco Customer Churn**, which details anonymous customers data of telecommunication company. Use the details of this data set to predict customer churn which is very critical to business as it's easier to retain existing customers rather than acquiring new ones.

## Learning goals

The learning goals of this notebook are:

-  Load a CSV file into an Apache® Spark DataFrame.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create an Apache® Spark machine learning pipeline.
-  Train and evaluate a model.
-  Persist a pipeline and model in Watson Machine Learning repository.
-  Explore and visualize prediction result using the plotly package.
-  Deploy a model for batch scoring using Wastson Machine Learning API.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Load and explore data](#load)
3.	[Create spark ml model](#model)
4.	[Persist model](#persistence)
5.	[Predict locally and visualize](#visualization)
6.	[Deploy and score in a Cloud](#scoring)
7.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a [Watson Machine Learning Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance required for model storing and deployment (a free plan is offered).
- Create a [Cloud Object Storage](https://console.bluemix.net/catalog/infrastructure/cloud-object-storage) instance required for storing scoring data (a free plan is offered).
- Create a [Apache Spark Service](https://console.bluemix.net/catalog/services/apache-spark/) instance required for batch scoring (a free plan is offered).
-  Make sure that you are using a Spark 2.0 kernel in notebook.

### Create the Telco Customer Churn Data Asset  

The Telco Customer Churn data is available on project's Github page.

1.  Run following cells to download dataset from **Telco Customer Churn project** page [data folder](https://github.com/pmservice/wml-sample-models/tree/master/spark/customer-satisfaction-prediction/data) to local machine filesystem.
2.  After downloading the dataset to your local machine, read it into Spark DataFrame.

In [1]:
# The code was removed by DSX for sharing.

Install wget package if necessary by running following command <br>
!pip install wget

In [2]:
import wget

link_to_data = 'https://github.com/pmservice/wml-sample-models/raw/master/spark/customer-satisfaction-prediction/data/WA_Fn%20UseC_%20Telco%20Customer%20Churn.csv'
filename = wget.download(link_to_data)

print(filename)

WA_Fn%20UseC_%20Telco%20Customer%20Churn.csv


<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache® Spark DataFrame and perform a basic exploration. 

Once you see the file on **Files** section, you can insert it to your notebook by selecting **Insert SparkSession DataFrame**

In [3]:
spark = SparkSession.builder.getOrCreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .option('nanValue', ' ')\
  .option('nullValue', ' ')\
  .load(filename)

Explore the loaded data by using the following Apache® Spark DataFrame methods:
-  print schema
-  count all records
-  show distribution of label classes

In [4]:
df_data.printSchema()

print("Number of fields: %3g" % len(df_data.schema))

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- Churn: string (nullable = true)

Number of fields:  21


As you can see, the data contains 21 fields. "Churn" field is the one we would like to predict (label).

In [5]:
print("Total number of records: " + str(df_data.count()))

Total number of records: 7043


Data set contains 7043 records.

Now you will check if all records have complete data.

In [6]:
df_complete = df_data.dropna()

print("Number of records with complete data: %3g" % df_complete.count())

Number of records with complete data: 7032


You can see that there are some missing values you can investigate that all missing values are present in `TotalCharges` feature. We will use dataset with missing values removed for model training and evaluation.

Now you will inspect distribution of classes in label column.

In [7]:
df_complete.groupBy('Churn').count().show()

+-----+-----+
|Churn|count|
+-----+-----+
|   No| 5163|
|  Yes| 1869|
+-----+-----+



<a id="model"></a>
## 3. Create an Apache® Spark machine learning model

In this section you will learn how to prepare data, create an Apache® Spark machine learning pipeline, and train a model.

### 3.1: Prepare data

In this subsection you will split your data into: train, test and predict datasets.

In [8]:
(train_data, test_data, predict_data) = df_complete.randomSplit([0.8, 0.18, 0.02], 24)

print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))
print("Number of records for prediction: " + str(predict_data.count()))

Number of records for training: 5621
Number of records for evaluation: 1263
Number of records for prediction: 148


As you can see our data has been successfully split into three datasets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.
-  The predict data set will be used for prediction.

### 3.2: Create pipeline and train a model

In this section you will create an Apache® Spark machine learning pipeline and then train the model.

In the first step you need to import the Apache® Spark machine learning packages that will be needed in the subsequent steps.

In [9]:
from pyspark.ml.feature import StringIndexer, IndexToString, RFormula
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, convert all the predictors to features vector and label feature convert to numeric.

In [10]:
lab = StringIndexer(inputCol = 'Churn', outputCol = 'label')
features = RFormula(formula = "~ gender + SeniorCitizen +  Partner + Dependents + tenure + PhoneService + MultipleLines + InternetService + OnlineSecurity + OnlineBackup + DeviceProtection + TechSupport + StreamingTV + StreamingMovies + Contract + PaperlessBilling + PaymentMethod + MonthlyCharges + TotalCharges - 1")

Next, define estimators you want to use for classification. Logistic Regression is used in the following example.

In [11]:
lr = LogisticRegression(maxIter = 10)

Let's build the pipeline now. A pipeline consists of transformers and an estimator.

In [12]:
pipeline_lr = Pipeline(stages = [features, lab , lr])

Now, you can train your Logistic Regression model by using the previously defined **pipeline** and **train data**.

In [13]:
model_lr = pipeline_lr.fit(train_data)

You can check your **model accuracy** now. To evaluate the model, use **test data**.

In [14]:
predictions = model_lr.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Test dataset:")
print("Accuracy = %3.2f" % accuracy)

Test dataset:
Accuracy = 0.81


You can tune your model now to achieve better accuracy. For simplicity of this example tuning section is omitted.

<a id="persistence"></a>
## 4. Persist model

In this section you will learn how to store your pipeline and model in Watson Machine Learning repository by using python client libraries.

First, you must import Watson Machine Learning client libraries.

**Note**: Apache® Spark 2.1 is required.

In [15]:
!rm -rf $PIP_BUILD/watson-machine-learning-client

In [16]:
!pip install watson-machine-learning-client --upgrade

    100% |████████████████████████████████| 215kB 1.7MB/s eta 0:00:01
Requirement already up-to-date: requests in /usr/local/src/bluemix_jupyter_bundle.v86/notebook/lib/python2.7/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: urllib3 in /usr/local/src/bluemix_jupyter_bundle.v86/notebook/lib/python2.7/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: pandas in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s86b-18e61b28c674e4-3fbaf243aed6/.local/lib/python2.7/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: certifi in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s86b-18e61b28c674e4-3fbaf243aed6/.local/lib/python2.7/site-packages (from watson-machine-learning-client)
    100% |████████████████████████████████| 61kB 1.0MB/s eta 0:00:01
Requirement already up-to-date: lomond in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s86b-18e61b28c674e4-3fbaf243aed6/.local/lib/python2.7/site-pack

In [17]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

Authenticate to Watson Machine Learning service on Bluemix.

**Action**: Put credentials from your instance of Watson Machine Learning service in Bluemix here.</div>

In [16]:
wml_credentials = {
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "***",
  "username": "***",
  "password": "***",
  "instance_id": "***"
}

In [18]:
# The code was removed by DSX for sharing.

In [19]:
client = WatsonMachineLearningAPIClient(wml_credentials)

### 4.1: Save pipeline and model

In this subsection you will learn how to save pipeline and model artifacts to your Watson Machine Learning instance.

In [22]:
saved_model = client.repository.store_model(model=model_lr, meta_props={'name':'my spark model'}, training_data=train_data, pipeline=pipeline_lr)

Get saved model metadata from Watson Machine Learning.

In [23]:
published_model_ID = client.repository.get_model_uid(saved_model)

print("Model Id: " + str(published_model_ID))

Model Id: 0da45aa2-5378-4dc1-8617-3d209499f466


**Model Id** can be used to retrive latest model version from Watson Machine Learning instance.

### 4.2: Load model

In this subsection you will learn how to load back saved model from specified instance of Watson Machine Learning.

In [24]:
loaded_model = client.repository.load(published_model_ID)

In [25]:
print(type(loaded_model))

<class 'pyspark.ml.pipeline.PipelineModel'>


As you can see the name is correct. You have already learned how save and load the model from Watson Machine Learning repository.

<a id="visualization"></a>
## 5. Predict locally and visualize

In this section you will learn how to load data from batch scoring and visualize the prediction results with plotly package.

### 5.1: Make local prediction using previously loaded model and test data

In this subsection you will score *predict_data* data set.

In [26]:
predictions = loaded_model.transform(predict_data)

Preview the results by calling the *show()* method on the predictions DataFrame.

In [27]:
predictions.show(5)

+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+--------------------+-----+--------------------+--------------------+----------+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|            features|label|       rawPrediction|         probability|prediction|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+----------------

By tabulating a count, you can see the split between labels.

In [28]:
predictions.select("prediction").groupBy("prediction").count().show(truncate=False)

+----------+-----+
|prediction|count|
+----------+-----+
|0.0       |114  |
|1.0       |34   |
+----------+-----+



### 5.2: Sample visualization of data with Plotly package

In this subsection you will explore prediction results with Plotly, which is an online analytics and data visualization tool.

**Example**: First, you need to install required packages. You can do it by running the following code. Run it only one time.

    !pip install plotly
    !pip install cufflinks

In [32]:
import sys
import pandas
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import plotly.graph_objs as go

init_notebook_mode(connected=True)
sys.path.append("".join([os.environ["HOME"]])) 

Plot a pie chart that shows the predicted churn split. !!! By most important features: Contract, MonthlyCharges !!! predicted vs observed churns.

In [33]:
cumulative_stats = predictions.groupby(['label']).count()
labels_data_plot = ['No','Yes']
values_data_plot = [cumulative_stats.select('count').collect()[x][0] for x in range(2)]

In [34]:
product_data = [go.Pie(
            labels=labels_data_plot,
            values=values_data_plot,
    )]

product_layout = go.Layout(
    title='Churn',
)

fig = go.Figure(data=product_data, layout=product_layout)
iplot(fig)

With this data set, you might want to do some analysis of Mean Monthly Charges per churn class

In [35]:
y_data_plot = [predictions.groupby(['label']).mean().collect()[x][3] for x in range(2)]

In [36]:
age_data = [go.Bar(
            y= y_data_plot,
            x= labels_data_plot
            
    )]

age_layout = go.Layout(
    title='Mean Monthly Charges per churn class',
    xaxis=dict(
        title = "Churn",
        showline=False,),
    yaxis=dict(
        title = "Mean Monthly Charges",
        ),
)

fig = go.Figure(data=age_data, layout=age_layout)
iplot(fig)

Based on bar plot you created, you might make the following conclusion: The mean monthly charges for churn customers is higher than non-churn customers as you may expect.

<a id="scoring"></a>
## 6. Deploy model and score in a Cloud

In this section you will learn how to create batch deployment and to score a new data record by using the Watson Machine Learning REST API. 
For more information about REST APIs, see the [Swagger Documentation](http://watson-ml-api.mybluemix.net/).

#### 6.1 Save data for prediction to Object Storage

ibm-cos-sdk library allows Python developers to manage Cloud Object Storage (COS).

In [29]:
import ibm_boto3
from ibm_botocore.client import Config

<div>**Action**: Put credentials from Object Storage Service in Bluemix here.</div>

In [101]:
cos_credentials = {
  "apikey": "***",
  "cos_hmac_keys": {
    "access_key_id": "***",
    "secret_access_key": "***"
  },
  "endpoints": "***",
  "iam_apikey_description": "***",
  "iam_apikey_name": "***",
  "iam_role_crn": "***",
  "iam_serviceid_crn": "***",
  "resource_instance_id": "***"
}

In [64]:
connection_apikey = cos_credentials['apikey']
connection_resource_instance_id = cos_credentials["resource_instance_id"]
connection_access_key_id = cos_credentials['cos_hmac_keys']['access_key_id']
connection_secret_access_key = cos_credentials['cos_hmac_keys']['secret_access_key']

**Action**: Define the service endpoint we will use. <br>
**Tip**: You can find this information in Endpoints section of your Cloud Object Storage intance's dashbord.

In [32]:
service_endpoint = 'https://s3-api.us-geo.objectstorage.softlayer.net'

You also need IBM Cloud authorization endpoint to be able to create COS resource object.

In [33]:
auth_endpoint = 'https://iam.bluemix.net/oidc/token'

We create COS resource to be able to write data to Cloud Object Storage.

In [34]:
cos = ibm_boto3.resource('s3',
                         ibm_api_key_id=cos_credentials['apikey'],
                         ibm_service_instance_id=cos_credentials['resource_instance_id'],
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

Now you will create bucket in COS and copy [dataset](https://raw.githubusercontent.com/pmservice/wml-sample-models/master/spark/customer-satisfaction-prediction/data/scoreInput.csv) for batch scoring from **Telco Customer Churn project**.

In [35]:
from uuid import uuid4

bucket_uid = str(uuid4())

score_filename = "scoreInput.csv"
buckets = ["batch-job-" + bucket_uid]

In [36]:
for bucket in buckets:
    if not cos.Bucket(bucket) in cos.buckets.all():
        print('Creating bucket "{}"...'.format(bucket))
        try:
            cos.create_bucket(Bucket=bucket)
        except ibm_boto3.exceptions.ibm_botocore.client.ClientError as e:
            print('Error: {}.'.format(e.response['Error']['Message']))

Creating bucket "batch-job-6595f0b0-182e-4bbd-b795-ea6909a183f5"...


In [37]:
data_links = ['https://raw.githubusercontent.com/pmservice/wml-sample-models/master/spark/customer-satisfaction-prediction/data/scoreInput.csv']

In [38]:
from urllib import urlopen

bucket_obj = cos.Bucket(buckets[0])

for data_link in data_links:
    filename=data_link.split('/')[-1]
    print('Uploading data {}...'.format(filename))
    scoring_data_connection = urlopen(data_link)
    bucket_obj.upload_fileobj(scoring_data_connection, filename)
    print('{} is uploaded.'.format(filename))

Uploading data scoreInput.csv...
scoreInput.csv is uploaded.


### 6.2: Create model deployment

Now you can create a batch scoring endpoint. Execute the following sample code that uses the **published Model Id** value to create the scoring endpoint to the Bluemix repository.

#### Create access token to Watson Machine Learning Service

To work with the Watson Machine Leraning REST API you must generate an access token. To do that you can use the following sample code:

First, you must import standard python libraries.

In [39]:
import urllib3, requests, json

headers = urllib3.util.make_headers(basic_auth='{username}:{password}'.format(username=wml_credentials['username'], password=wml_credentials['password']))
url = '{url}/v3/identity/token'.format(url=wml_credentials['url'])
response = requests.get(url, headers=headers)
mltoken = json.loads(response.text).get('token')

### 6.1: Create batch deployment

#### Get published_models url from instance details

In [41]:
endpoint_instance = wml_credentials['url'] + "/v3/wml_instances/" + wml_credentials['instance_id']
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

response_get_instance = requests.get(endpoint_instance, headers=header)

print(response_get_instance)
print(json.dumps(response_get_instance.text, indent=2))

<Response [200]>
"{\"metadata\":{\"guid\":\"37c88e6f-2a51-43f8-be5a-17a928580075\",\"url\":\"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/37c88e6f-2a51-43f8-be5a-17a928580075\",\"created_at\":\"2018-01-04T14:17:28.939Z\",\"modified_at\":\"2018-02-15T09:24:19.976Z\"},\"entity\":{\"source\":\"Bluemix\",\"published_models\":{\"url\":\"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/37c88e6f-2a51-43f8-be5a-17a928580075/published_models\"},\"usage\":{\"expiration_date\":\"2018-03-01T00:00:00.000Z\",\"computation_time\":{\"limit\":18000,\"current\":0},\"model_count\":{\"limit\":200,\"current\":125},\"prediction_count\":{\"limit\":5000,\"current\":0},\"deployment_count\":{\"limit\":5,\"current\":0}},\"plan_id\":\"3f6acf43-ede8-413a-ac69-f8af3bb0cbfe\",\"status\":\"Active\",\"organization_guid\":\"b0e61605-a82e-4f03-9e9f-2767973c084d\",\"region\":\"us-south\",\"account\":{\"id\":\"f52968f3dbbe7b0b53e15743d45e5e90\",\"name\":\"Umit Cakmak's Account\",\"type\":\"TRIAL\"},\"owner\":{

In [42]:
endpoint_published_models = json.loads(response_get_instance.text).get('entity').get('published_models').get('url')

print(endpoint_published_models)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/37c88e6f-2a51-43f8-be5a-17a928580075/published_models


Execute the following sample code that uses the published_models endpoint to get deployments url.

#### Get the list of published models

In [43]:
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}
response_get = requests.get(endpoint_published_models, headers=header)

print(response_get)
print(json.dumps(response_get.text, indent=2))

<Response [200]>
"{\"count\":1,\"resources\":[{\"metadata\":{\"guid\":\"b7655b89-657a-4f88-bf80-0478bcd2a678\",\"url\":\"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/37c88e6f-2a51-43f8-be5a-17a928580075/published_models/b7655b89-657a-4f88-bf80-0478bcd2a678\",\"created_at\":\"2018-02-15T09:20:42.995Z\",\"modified_at\":\"2018-02-15T09:20:43.111Z\"},\"entity\":{\"runtime_environment\":\"spark-2.0\",\"learning_configuration_url\":\"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/37c88e6f-2a51-43f8-be5a-17a928580075/published_models/b7655b89-657a-4f88-bf80-0478bcd2a678/learning_configuration\",\"author\":{},\"name\":\"Customer Churn Model\",\"label_col\":\"Churn\",\"learning_iterations_url\":\"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/37c88e6f-2a51-43f8-be5a-17a928580075/published_models/b7655b89-657a-4f88-bf80-0478bcd2a678/learning_iterations\",\"training_data_schema\":{\"fields\":[{\"metadata\":{},\"type\":\"string\",\"name\":\"customerID\",\"nullable\":true},{\"me

#### Get published model deployment url

In [44]:
[endpoint_deployments] = [x.get('entity').get('deployments').get('url') for x in json.loads(response_get.text).get('resources') if x.get('metadata').get('guid') == saved_model.uid]

print(endpoint_deployments)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/37c88e6f-2a51-43f8-be5a-17a928580075/published_models/b7655b89-657a-4f88-bf80-0478bcd2a678/deployments


#### Create batch deployment for published model

First import standard python library.

In [45]:
import base64

<div>**Action**:  Put your credentials from Apache Spark Service in Bluemix here</div>

In [102]:
spark_credentials = {
  "tenant_id": "***",
  "tenant_id_full": "***",
  "cluster_master_url": "***",
  "tenant_secret": "***",
  "instance_id": "***",
  "plan": "***"
}

In [47]:
encoded_spark_credentials_header = base64.b64encode(json.dumps({"credentials": spark_credentials, "version": "2.0"}))
result_filename = "result.csv"

In [103]:
header_batch = {'Content-Type': 'application/json', 'Authorization': "Bearer " + mltoken, 'X-Spark-Service-Instance': encoded_spark_credentials_header}
payload_batch = {"type":"batch", "name": "Customer Churn Prediction", "description": "Batch Deployment", "input": {"source": { "bucket": buckets[0], "filename": score_filename, "inferschema":1,  "fileformat": "csv", "type": "cloudobjectstorage"}, "connection": {"access_key": connection_access_key_id, "secret_key": connection_secret_access_key, "url": service_endpoint}}, "output": { "target": { "bucket": buckets[0], "filename": result_filename, "fileformat": "csv", "firstlineheader":"true", "type": "cloudobjectstorage"}, "connection": {"access_key": connection_access_key_id, "secret_key": connection_secret_access_key, "url": service_endpoint}}}

response_batch = requests.post(endpoint_deployments, json=payload_batch, headers=header_batch)

print(response_batch)
print(json.loads(response_batch.text).get("entity").get("status"))

<Response [201]>
INITIALIZING


You can see that your job has been submited to Spark.

#### Monitor job execution

Here you can check status of your batch scoring. When status of Spark job is `FINNISHED` the results will be written to result.csv file in Object Storage.

In [104]:
header_monitor_job = {'Content-Type': 'application/json', 'Authorization': "Bearer " + mltoken, 'X-Spark-Service-Instance': encoded_spark_credentials_header}

response_monitor_job = requests.get(endpoint_deployments, headers=header_monitor_job)

print(response_monitor_job)
print(json.dumps(response_monitor_job.text, indent=2))

<Response [200]>
"{\"count\":1,\"resources\":[{\"metadata\":{\"guid\":\"2a62e2b0-3504-4a08-ae35-9090eb9e6c3a\",\"url\":\"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/37c88e6f-2a51-43f8-be5a-17a928580075/published_models/b7655b89-657a-4f88-bf80-0478bcd2a678/deployments/2a62e2b0-3504-4a08-ae35-9090eb9e6c3a\",\"created_at\":\"2018-02-15T10:17:11.637Z\",\"modified_at\":\"2018-02-15T10:17:32.112Z\"},\"entity\":{\"runtime_environment\":\"spark-2.0\",\"name\":\"Customer Churn Prediction\",\"description\":\"Batch Deployment\",\"published_model\":{\"author\":{},\"name\":\"Customer Churn Model\",\"url\":\"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/37c88e6f-2a51-43f8-be5a-17a928580075/published_models/b7655b89-657a-4f88-bf80-0478bcd2a678\",\"guid\":\"b7655b89-657a-4f88-bf80-0478bcd2a678\",\"created_at\":\"2018-02-15T09:25:53.441Z\"},\"model_type\":\"mllib-2.0\",\"status\":\"COMPLETED\",\"status_message\":\"SUCCESS\",\"output\":{\"connection\":{\"access_key\":\"fe8d4a595ab8432bb8

**Preview scored data**

In this subsection you will load scored data.

Get data with scores from Cloud Object Storage.

In [105]:
bucket_obj.download_file(Key='result.csv',Filename='result.csv')

'result.csv' file is downloaded to your local file directory

In [106]:
ls res*

result.csv


Loading 'result.csv' file as pandas dataframe

In [109]:
import pandas as pd
results_pdf = pd.read_csv('result.csv')

Preview the results by calling the *head()* method on the predictions in Pandas data frame.

In [110]:
results_pdf.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,PaymentMethod,MonthlyCharges,TotalCharges,Churn,SampleWeight,features,label,rawPrediction,probability,prediction
0,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,Electronic check,70.7,151.65,Yes,1.000,"(30,[2,3,4,5,6,8,10,12,14,16,18,20,22,24,25,28...",1,"[-0.6683985606091145,0.6683985606091145]","[0.3388555225599606,0.6611444774400395]",1
1,3638-WEABW,Female,0,Yes,No,58,Yes,Yes,DSL,No,...,Credit card (automatic),59.9,3505.10,No,2.768,"(30,[3,4,5,7,9,10,13,14,17,18,20,23,24,28,29],...",0,"[3.94407920167498,-3.94407920167498]","[0.9809989882724504,0.019001011727549564]",0
2,8665-UTDHZ,Male,0,Yes,Yes,1,No,No phone service,DSL,No,...,Electronic check,30.2,30.20,Yes,1.000,"(30,[0,4,9,10,13,14,16,18,20,22,25,28,29],[1.0...",1,"[-0.09214192316229042,0.09214192316229042]","[0.4769808032441,0.5230191967558999]",1
3,8773-HHUOZ,Female,0,No,Yes,17,Yes,No,DSL,No,...,Mailed check,64.7,1093.10,Yes,1.000,"(30,[2,4,5,6,9,10,12,14,16,19,21,22,24,26,28,2...",1,"[0.747504546477989,-0.747504546477989]","[0.6786347093927774,0.32136529060722263]",0
4,4080-IIARD,Female,0,Yes,No,13,Yes,Yes,DSL,Yes,...,Electronic check,76.2,981.45,No,2.768,"(30,[3,4,5,7,9,11,13,14,17,19,20,22,24,25,28,2...",0,"[0.5917942689227103,-0.5917942689227103]","[0.6437767283180145,0.3562232716819854]",0


You can see the predictions for all the records in pandas data frame.

<a id="summary"></a>
## 7. Summary and next steps     

 You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. Check out our _[Online Documentation](https://console.ng.bluemix.net/docs/services/PredictiveModeling/index.html?pos=2)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Umit Cakmak**, Data Scientist in IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.